In [1]:
from multiprocessing import Process, Pool
from tqdm import trange
import numpy as np

from particle.pipeline import Sand

from numpy.linalg.linalg import LinAlgError

In [2]:
trainSet = np.load('data/liutao/v1/particles.npz')['trainSet']
testSet = np.load('data/liutao/v1/particles.npz')['testSet']

## 颗粒几何特征
- volume
- surface
- radius
- sphericity
- EI, FI
- convexity
- angularity
- roughness

In [3]:
def cal(particles, name, error):
    threshold = 0.5
    particles[particles <= threshold] = 0
    particles[particles > threshold] = 1
    
    volume = np.empty(len(particles), dtype=np.float32)
    surface = np.empty(len(particles), dtype=np.float32)
    radius = np.empty(len(particles), dtype=np.float32)
    sphericity = np.empty(len(particles), dtype=np.float32)
    EI = np.empty(len(particles), dtype=np.float32)
    FI = np.empty(len(particles), dtype=np.float32)
    convexity = np.empty(len(particles), dtype=np.float32)
    angularity = np.empty(len(particles), dtype=np.float32)
    roughness = np.empty(len(particles), dtype=np.float32)

    for i in trange(len(particles)):
        try:
            cube = particles[i, 0]
            sand = Sand(cube)
            volume[i] = np.sum(cube == 1)
            surface[i] = sand.surf_area()
            radius[i] = sand.circumscribed_sphere()[0]
            sphericity[i] = sand.sphericity()
            EI[i], FI[i] = sand.EI_FI()
            convexity[i] = sand.convexity()
            angularity[i] = sand.angularity()
            roughness[i] = sand.roughness()
        except (RuntimeError, LinAlgError):
            error.append(i)
            continue

    mask = np.ones(len(particles), dtype=bool)
    mask[error] = False
    np.savez(f'output/{name}.npz',
             volume=volume[mask],
             surface=surface[mask],
             radius=radius[mask],
             sphericity=sphericity[mask],
             EI=EI[mask], FI=FI[mask],
             convexity=convexity[mask],
             angularity=angularity[mask],
             roughness=roughness[mask],
             mask=mask)
    return error

In [4]:
errorTrain = []
errorTest = []

In [5]:
pTrain = Process(target=cal, args=(trainSet, 'trainSet', errorTrain))
pTest = Process(target=cal, args=(testSet, 'testSet', errorTest))

In [6]:
pTrain.start()

100%|██████████| 27143/27143 [2:02:30<00:00,  3.69it/s]  
/opt/miniconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [7]:
pTest.start()

100%|██████████| 6786/6786 [36:30<00:00,  3.10it/s]  
/opt/miniconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [8]:
pTest.join()
print("testSet has been finished...")
pTrain.join()
print("trainSet has been finished...")

testSet has been finished...
trainSet has been finished...


In [9]:
print(len(errorTrain), len(errorTest))

0 0


In [10]:
errorTrain

[]

In [11]:
pTrain.is_alive()

False